<a href="https://colab.research.google.com/github/seuhihihi/Colab-Python/blob/main/%E6%B7%B7%E5%90%88%E6%B0%94%E4%BD%93%E5%8E%8B%E7%BC%A9%E7%B3%BB%E6%95%B0%E8%AE%A1%E7%AE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# 安装所需库（如果尚未安装）
!pip install pandas openpyxl

import numpy as np
import pandas as pd

def peng_robinson_with_kij_mass(temp_C, pressure_bar, components, volume_fractions, excel_filename=None):
    R = 8.314  # J/mol/K
    T = temp_C + 273.15
    P_total = pressure_bar * 1e5

    # 临界属性数据库（Tc [K], Pc [Pa], omega）
    critical_data = {
        "CH4": [190.6, 4.60e6, 0.011], "C2H6": [305.3, 4.88e6, 0.099],
        "C2H4": [282.3, 5.04e6, 0.086], "C3H8": [369.8, 4.25e6, 0.152],
        "C3H6": [364.9, 4.66e6, 0.142], "i-C4H10": [407.7, 3.64e6, 0.184],
        "n-C4H10": [425.2, 3.80e6, 0.200], "t-2-C4H8": [420.0, 4.30e6, 0.225],
        "1-C4H8": [419.4, 4.25e6, 0.200], "i-C4H8": [415.0, 4.15e6, 0.197],
        "c-2-C4H8": [417.0, 4.20e6, 0.200], "1,3-C4H6": [440.0, 4.20e6, 0.200],
        "H2": [33.2, 1.30e6, -0.220], "He": [5.2, 0.23e6, -0.390],
        "N2": [126.2, 3.39e6, 0.037]
    }

    kij_defaults = {
        ("CH4", "C2H6"): 0.001, ("CH4", "C2H4"): 0.03, ("CH4", "C3H8"): 0.01,
        ("CH4", "C3H6"): 0.02, ("CH4", "N2"): 0.025, ("CH4", "H2"): 0.02,
        ("CH4", "He"): 0.0, ("C2H6", "C3H8"): 0.001, ("C2H6", "N2"): 0.03,
        ("C2H6", "H2"): 0.035, ("C2H6", "He"): 0.005, ("H2", "He"): 0.0,
        ("H2", "N2"): 0.06, ("N2", "He"): 0.02, ("C3H6", "N2"): 0.025,
    }

    molar_masses = {
        "CH4": 16.04246, "C2H6": 30.06904, "C2H4": 28.05316, "C3H8": 44.09562, "C3H6": 42.07974,
        "i-C4H10": 58.1222, "n-C4H10": 58.1222, "t-2-C4H8": 56.10632, "1-C4H8": 56.10632,
        "i-C4H8": 56.10632, "c-2-C4H8": 56.10632, "1,3-C4H6": 54.09044, "H2": 2.01588,
        "He": 4.002602, "N2": 28.0134
    }

    n = len(components)
    a_i = np.zeros(n)
    b_i = np.zeros(n)
    Z_i = np.zeros(n)
    partial_pressures = np.array(volume_fractions) * P_total
    molar_mass_arr = np.array([molar_masses[c] for c in components]) / 1000  # kg/mol

    for i, comp in enumerate(components):
        Tc, Pc, omega = critical_data[comp]
        Tr = T / Tc
        m = 0.37464 + 1.54226 * omega - 0.26992 * omega**2
        alpha = (1 + m * (1 - np.sqrt(Tr)))**2
        a_i[i] = 0.45724 * R**2 * Tc**2 / Pc * alpha
        b_i[i] = 0.07780 * R * Tc / Pc

        P_i = partial_pressures[i]
        A = a_i[i] * P_i / (R**2 * T**2)
        B = b_i[i] * P_i / (R * T)
        coeffs = [1, B - 1, A - 3 * B**2 - 2 * B, -(A * B - B**2 - B**3)]
        Z_real = np.roots(coeffs)[np.isreal(np.roots(coeffs))].real
        Z_i[i] = max(Z_real)

    raw_ratio = volume_fractions / Z_i
    mole_fractions_corrected = raw_ratio / np.sum(raw_ratio)

    a_mix = 0
    for i in range(n):
        for j in range(n):
            key = tuple(sorted([components[i], components[j]]))
            kij = kij_defaults.get(key, 0.0)
            a_mix += mole_fractions_corrected[i] * mole_fractions_corrected[j] * np.sqrt(a_i[i] * a_i[j]) * (1 - kij)
    b_mix = np.sum(mole_fractions_corrected * b_i)

    A = a_mix * P_total / (R**2 * T**2)
    B = b_mix * P_total / (R * T)
    coeffs_mix = [1, B - 1, A - 3 * B**2 - 2 * B, -(A * B - B**2 - B**3)]
    Z_mix = max(np.roots(coeffs_mix)[np.isreal(np.roots(coeffs_mix))].real)

    C_total = P_total / (Z_mix * R * T)
    molar_concentrations = mole_fractions_corrected * C_total
    mass_concentrations = molar_concentrations * molar_mass_arr
    total_mass_concentration = np.sum(mass_concentrations)
    mass_fractions = mass_concentrations / total_mass_concentration

    if excel_filename:
        df = pd.DataFrame({
            "Component": components,
            "VolumeFraction": volume_fractions,
            "PartialPressure (Pa)": partial_pressures,
            "Z_i (at partial P)": Z_i,
            "MoleFraction": mole_fractions_corrected,
            "MolarConcentration (mol/m^3)": molar_concentrations,
            "MassConcentration (kg/m^3)": mass_concentrations,
            "MassFraction": mass_fractions
        })
        df.to_excel(excel_filename, index=False)
        print(f"Results written to {excel_filename}\\nK_ij source: Reid, Prausnitz, Poling, 1987.")

    # 打印输出
    print(f"压缩因子 Z_mix: {Z_mix:.4f}")
    print(f"总摩尔浓度: {C_total:.2f} mol/m³")
    print(f"总质量浓度: {total_mass_concentration:.4f} kg/m³\\n")

    print("各组分浓度与分数:")
    for i in range(n):
        print(f"{components[i]:<10} Mole Frac: {mole_fractions_corrected[i]:.6f}, "
              f"Molar Conc.: {molar_concentrations[i]:.2f} mol/m³, "
              f"Mass Conc.: {mass_concentrations[i]:.4f} kg/m³, "
              f"Mass Frac.: {mass_fractions[i]:.6f}")

    return Z_mix, C_total, mole_fractions_corrected, molar_concentrations, mass_concentrations, mass_fractions, total_mass_concentration

# 示例运行
components = ['CH4','C2H6','C2H4','C3H8','C3H6','i-C4H10','n-C4H10',
              't-2-C4H8','1-C4H8','i-C4H8','c-2-C4H8','1,3-C4H6','H2','He','N2']
volume_fractions = [1,0.3,7,0.5,5.5,0.1,0.1,0.3,0.3,0.5,0.25,0.01,0.1,1,83.04]
volume_fractions = [v / sum(volume_fractions) for v in volume_fractions]

Z_mix, C_total, mole_fractions, molar_concentrations, \
mass_concentrations, mass_fractions, total_mass_concentration = peng_robinson_with_kij_mass(
    temp_C=15,
    pressure_bar=150,
    components=components,
    volume_fractions=volume_fractions,
    excel_filename="gas_mixture_results.xlsx"
)

print(f"Z_mix = {Z_mix:.4f}")
print(f"Total molar concentration = {C_total:.2f} mol/m³")
from google.colab import files
files.download('gas_mixture_results.xlsx')

Results written to gas_mixture_results.xlsx\nK_ij source: Reid, Prausnitz, Poling, 1987.
压缩因子 Z_mix: 0.9003
总摩尔浓度: 6954.49 mol/m³
总质量浓度: 201.9687 kg/m³\n
各组分浓度与分数:
CH4        Mole Frac: 0.009763, Molar Conc.: 67.89 mol/m³, Mass Conc.: 1.0892 kg/m³, Mass Frac.: 0.005393
C2H6       Mole Frac: 0.002930, Molar Conc.: 20.38 mol/m³, Mass Conc.: 0.6127 kg/m³, Mass Frac.: 0.003034
C2H4       Mole Frac: 0.073790, Molar Conc.: 513.17 mol/m³, Mass Conc.: 14.3960 kg/m³, Mass Frac.: 0.071279
C3H8       Mole Frac: 0.004930, Molar Conc.: 34.29 mol/m³, Mass Conc.: 1.5119 kg/m³, Mass Frac.: 0.007486
C3H6       Mole Frac: 0.062555, Molar Conc.: 435.04 mol/m³, Mass Conc.: 18.3063 kg/m³, Mass Frac.: 0.090639
i-C4H10    Mole Frac: 0.000977, Molar Conc.: 6.79 mol/m³, Mass Conc.: 0.3948 kg/m³, Mass Frac.: 0.001955
n-C4H10    Mole Frac: 0.000977, Molar Conc.: 6.79 mol/m³, Mass Conc.: 0.3949 kg/m³, Mass Frac.: 0.001955
t-2-C4H8   Mole Frac: 0.002952, Molar Conc.: 20.53 mol/m³, Mass Conc.: 1.1520 kg/m³, Mass Fr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>